Demostration of Mixture of expert ensemble method
- Training

  1. Cluster input data. Clustering results to train the gating network. 
  2. Assign each cluster to an expert classifier. Train each expert classifier. 

- Inferencing
  1. input a testing sample to the gating network. use the one-hot encoded output of the gating network to select an expert
  2. Input the same testing sample to the select expert and use the expert's prediction as the MoE prediction.

In [1]:
# Load Fisher's iris data set.
import numpy as np
from sklearn.model_selection import train_test_split
from google.colab import drive

drive.mount('/content/drive')
with open('/content/drive/My Drive/Colab Notebooks/data/iris.csv', 'r') as f: 
    tmp = np.genfromtxt(f, dtype='f4', delimiter=',')

X = tmp[:,0:-1]
Y = tmp[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, shuffle=None)

Mounted at /content/drive


In [2]:
# Gating Network initiation: Using kmeans algorithm to partition the data into 3 clusters
from sklearn.cluster import KMeans

C = 7
kmeans = KMeans(n_clusters=C, random_state=0).fit(X_train)
idx = kmeans.labels_

In [3]:
# Traing the gating network model using a naive baysian classifier using the clustering indices obtained after clustering
from sklearn.naive_bayes import GaussianNB

gNet = GaussianNB().fit(X_train, idx)

In [4]:
# Traing individual expert classifiers using each cluster of training data
from sklearn.tree import DecisionTreeClassifier

Exp = []
for k in range(C):
  Exp.append(DecisionTreeClassifier(random_state=0).fit(X_train[idx==k], y_train[idx==k]))

In [5]:
# Validation with X_test, y_test
from keras.utils.np_utils import to_categorical   

# Gating network output
y_gNet = gNet.predict(X_test)
y_gNet_1h = to_categorical(y_gNet)


# Let each expert predict for the entire validation data set
y_exp = []
for k in range(C):
  y_exp.append(Exp[k].predict(X_test))

# Selecting the prediction from corresponding expert assigned by the gating network
y_exp_T = np.array(y_exp).T
y_gated = np.sum(y_exp_T * y_gNet_1h, axis=1)

In [6]:
# Check results
from sklearn.metrics import confusion_matrix

# expert classifier's performance w.r.t. the entire validation dataset
for k in range(C):
  print(f'Expert #{k}\n',confusion_matrix(y_test,y_exp[k]))

print('Gated \n', confusion_matrix(y_test,y_gated))

Expert #0
 [[ 0 19  0]
 [ 0 14  1]
 [ 0  0 11]]
Expert #1
 [[19  0  0]
 [15  0  0]
 [11  0  0]]
Expert #2
 [[ 0  0 19]
 [ 0  0 15]
 [ 0  0 11]]
Expert #3
 [[ 0  0 19]
 [ 0  0 15]
 [ 0  0 11]]
Expert #4
 [[19  0  0]
 [15  0  0]
 [11  0  0]]
Expert #5
 [[ 0 19  0]
 [ 0 14  1]
 [ 0  0 11]]
Expert #6
 [[ 0 19  0]
 [ 0 15  0]
 [ 0 11  0]]
Gated 
 [[19  0  0]
 [ 0 14  1]
 [ 0  0 11]]


In [7]:
# find which expert made the wrong classification
print('expert #',y_gNet[y_gated != y_test],'made incorrect prediction')

expert # [5] made incorrect prediction


In [8]:
drive.flush_and_unmount()